In [ ]:
import pandas as pd
from typing import Optional, List, Tuple
import os

os.environ['TOKENIZERS_PARALLELISM'] = 'false' # do not use multiprocessing to avoid the paralelism problem
pd.set_option("display.max_colwidth", None)  # this will be helpful when visualizing retriever outputs

### 0. Load dataset

In [ ]:
from langchain.document_loaders import UnstructuredFileLoader

RAW_KNOWLEDGE_BASE = UnstructuredFileLoader("./docs/hobbit.txt").load()

### 1. Retriever - embeddings
<p>The retriever acts like an internal search engine: given the user query, it returns a few relevant snippets from your knowledge base. These snippets will then be fed to the Reader Model to help it generate its answer.</p>
How many snippets should we retrieve? This parameter will be named <b>top_k</b>. <br>
How long should these snippets be? This is called the <b>chunk size</b>.

<i>"In this notebook, we use Langchain library since it offers a huge variety of options for vector databases and allows us to keep document metadata throughout the processing."</i>

#### 1.1 Split the documents into chunks
- In this part, we split the documents from our knowledge base into smaller chunks which will be the snippets on which the reader LLM will base its answer.
- The goal is to prepare a collection of semantically relevant snippets. So their size should be adapted to precise ideas: too small will truncate ideas, too large will dilute them.

<i>Many options exist for text splitting: splitting on words, on sentence boundaries, recursive chunking that processes documents in a tree-like way to preserve structure information…</i>

- <b>Recursive chunking</b> breaks down the text into smaller parts step by step using a given list of separators sorted from the most important to the least important separator. <br>
For instance with the list of separators ["\n\n", "\n", ".", ""]:
    - The method will first break down the document wherever there is a double line break "\n\n".
    - Resulting documents will be split again on simple line breaks "\n", then on sentence ends ".".
    - And finally, if some chunks are still too big, they will be split whenever they overflow the maximum size.  

We also keep in mind that before we split the documents into chuncks we have to embed the documents and it could be a problem that the chunk lengths are not aligned with our limit maximum sequence length of the embedding model. </br>
<b>So we should make sure that our chunck sizes are below this limit, because any longer chunk will be trucated before processing, thus losing relevancy.</b>

In [ ]:
from sentence_transformers import SentenceTransformer

EMBED_MODEL_NAME = 'thenlper/gte-small'

# To get the value of the max sequence_length, we will query the underlying `SentenceTransformer` object used in the RecursiveCharacterTextSplitter: 512.
print(f"Model's maximum sequence length: {SentenceTransformer(EMBED_MODEL_NAME).max_seq_length}")

Now that we already know the maximum length of the embedding model we can create a RecursiveCharacterTextSplitter to count length in number of tokens instead of number of characters and then we can choose specific chunck size. To do that we indicate in the splitter the tokenizer we will use.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer
from langchain.docstore.document import Document as LangchainDocument

# This list is taken from LangChain's MarkdownTextSplitter class.
MARKDOWN_SEPARATORS = [
    "\n#{1,6} ",
    "```\n",
    "\n\\*\\*\\*+\n",
    "\n---+\n",
    "\n___+\n",
    "\n\n",
    "\n",
    " ",
    "",
]

def split_documents(chunk_size: int, knowledge_base: List[LangchainDocument], tokenizer_name: Optional[str] = EMBED_MODEL_NAME, ) -> List[LangchainDocument]:
    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        AutoTokenizer.from_pretrained(tokenizer_name), # we use the same tokenizer model as the embedding model
        chunk_size=chunk_size,  # the maximum number of characters in a chunk: we selected this value arbitrarily
        chunk_overlap=int(chunk_size/10),  # the number of characters to overlap between chunks
        add_start_index=True,  # If `True`, includes chunk's start index in metadata
        strip_whitespace=True,  # If `True`, strips whitespace from the start and end of every document
        separators=MARKDOWN_SEPARATORS,
    )

    docs_processed = []
    for doc in knowledge_base:
        docs_processed += text_splitter.split_documents([doc])

    # Remove duplicates
    unique_texts = {}
    docs_processed_unique = []
    for doc in docs_processed:
        if doc.page_content not in unique_texts:
            unique_texts[doc.page_content] = True
            docs_processed_unique.append(doc)

    return docs_processed_unique

docs_processed = split_documents(
    512,  # We choose a chunk size adapted to our model
    RAW_KNOWLEDGE_BASE,
    tokenizer_name=EMBED_MODEL_NAME,
)

#### 1.2 Building the vector database
We want to compute the embeddings for all the chunks of our knowledge base.</br> We first convert the chunks tokenized into embeddings.

##### <b>How does retrieval work?</b> </br>
Once the chunks are all embedded, we store them into a vector database. When the user types in a query, it gets embedded by the same model previously used, and a similarity search returns the closest documents from the vector database.

The technical challenge is thus, given a query vector, to quickly find the nearest neighbours of this vector in the vector database. <br>
To do this, we need to choose two things: a <b>distance</b>, and a <b>search algorithm</b> to find the nearest neighbors quickly within a database of thousands of records.

##### <b>Nearest Neighbor search algorithm</b>
There are plentiful choices for the nearest neighbor search algorithm: we go with Facebook’s <b>FAISS</b>, since FAISS is performant enough for most use cases, and it is well known thus widely implemented.

##### Distances
We will use <b>Cosine similarity</b>. <br> It computes similarity between two vectors as the cosinus of their relative angle: it allows us to compare vector directions are regardless of their magnitude. Using it requires to normalize all vectors, to rescale them into unit norm.

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
    model_name=EMBED_MODEL_NAME,
    multi_process=True, 
    model_kwargs={"device": "mps"},
    encode_kwargs={"normalize_embeddings": True},  # set True for cosine similarity
)

🚨👇 The cell below takes a few minutes to run on A10G!

In [ ]:
from langchain.vectorstores import Chroma

KNOWLEDGE_VECTOR_DATABASE = Chroma.from_documents(documents=docs_processed, embedding=embedding_model, persist_directory="./chroma")

We want to find the k document that have the closest meaning, thus we pick the k closest vectors.<br>
In the LangChain vector database implementation, this search operation is performed by the method vector_database.similarity_search(query).

### Questions

In [ ]:
# embed a user query in the same space
user_query = "How to create a pipeline object?"
query_vector = embedding_model.embed_query(user_query)

print(f"\nStarting retrieval for {user_query=}...")
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=5)
print("\n==================================Top document==================================")
print(retrieved_docs[0].page_content)
print("==================================Metadata top document==================================")
print(retrieved_docs[0].metadata)
print("==================================All documents==================================")
for doc in retrieved_docs:
    print(doc.metadata)

### Evaluate similarity
To evaluate and graphically display the similarity between the vector of your question and the vectors returned, you can use the concept of vector similarity measures such as cosine similarity. Here's a simple example using Python and the NumPy library. This code calculates the cosine similarity between the question vector and the returned vectors, then displays the similarity as a percentage. It also visualizes the vectors as arrows in a 2D plot.

In [ ]:
import numpy as np

# Assume these are the vectors returned from your data
vectors = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
# This is your question vector
question_vector = np.array([2, 3, 4])

# Calculate cosine similarity
cosine_similarities = []
for vector in vectors:
    dot_product = np.dot(vector, question_vector)
    norm_vector = np.linalg.norm(vector)
    norm_question_vector = np.linalg.norm(question_vector)
    similarity = dot_product / (norm_vector * norm_question_vector)
    cosine_similarities.append(similarity)

# Display the similarity as a percentage
for i, sim in enumerate(cosine_similarities):
    print(f"The similarity between the question vector and vector {i} is {sim * 100:.2f}%")

# For visualization, you can use a library like matplotlib
import matplotlib.pyplot as plt

# Plot the vectors as arrows in a 2D plot (you can extend this to 3D if needed)
questions_2d = question_vector.reshape(1, -1)
vectors_2d = vectors.reshape(-1, 2)  # assuming 2D vectors
all_vectors = np.concatenate((questions_2d, vectors_2d), axis=0)

fig, ax = plt.subplots()
ax.quiver(all_vectors[:, 0], all_vectors[:, 1], all_vectors[:, 0], all_vectors[:, 1], angles='xy', scale_units='xy', scale=1)
ax.set_xlim([-10, 10])
ax.set_ylim([-10, 10])
plt.show()

### 2. Reader - LLM
The LLM Reader reads the retrieved context (the content of the retrieved documents) to formulate its answer. <br>
The context and the user query are aggregated into a prompt then given to the LLM to generate its answer.

#### 2.1. Reader model
The choice of a reader model is important because...<br><br>
<b>The reader model’s max_seq_length must accomodate our prompt</b> , which includes the context output by the retriever call: the context consists in 5 documents of 512 tokens each, so we aim for a context length of 4k tokens at least.<br><br>
For this example, we chose HuggingFaceH4/zephyr-7b-beta, a small but powerful model.<br><br>
To make inference faster, we will load the quantized version of the model:

In [ ]:
from transformers import pipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

READER_MODEL_NAME = "HuggingFaceH4/zephyr-7b-beta"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)
model = AutoModelForCausalLM.from_pretrained(READER_MODEL_NAME, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(READER_MODEL_NAME)

READER_LLM = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=500,
)

#### 2.2. Prompt
The RAG prompt template below is what we will feed to the Reader LLM: it is important to have it formatted in the Reader LLM’s chat template.<br><br>

We give it our context and the user’s question.

In [ ]:
prompt_in_chat_format = [
    {
        "role": "system",
        "content": """Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.""",
    },
    {
        "role": "user",
        "content": """Context:
{context}
---
Now here is the question you need to answer.

Question: {question}""",
    },
]
RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(
    prompt_in_chat_format, tokenize=False, add_generation_prompt=True
)
print(RAG_PROMPT_TEMPLATE)

#### 2.3. Reranking
A good option for RAG is to retrieve more documents than you want in the end, then rerank the results with a more powerful retrieval model before keeping only the top_k.<br><br>

For this, Colbertv2 is a great choice: instead of a bi-encoder like our classical embedding models, it is a cross-encoder that computes more fine-grained interactions between the query tokens and each document’s tokens.<br><br>

It is easily usable thanks to the RAGatouille library.<br><br>
Let’s test our Reader on our previously retrieved documents!

In [ ]:
from ragatouille import RAGPretrainedModel
from transformers import Pipeline

RERANKER = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

def answer_with_rag(
    question: str,
    llm: Pipeline,
    knowledge_index: FAISS,
    reranker: Optional[RAGPretrainedModel] = None,
    num_retrieved_docs: int = 30,
    num_docs_final: int = 5,
) -> Tuple[str, List[LangchainDocument]]:
    # Gather documents with retriever
    print("=> Retrieving documents...")
    relevant_docs = knowledge_index.similarity_search(query=question, k=num_retrieved_docs)
    relevant_docs = [doc.page_content for doc in relevant_docs]  # keep only the text

    # Optionally rerank results
    if reranker:
        print("=> Reranking documents...")
        relevant_docs = reranker.rerank(question, relevant_docs, k=num_docs_final)
        relevant_docs = [doc["content"] for doc in relevant_docs]

    relevant_docs = relevant_docs[:num_docs_final]

    # Build the final prompt
    context = "\nExtracted documents:\n"
    context += "".join([f"Document {str(i)}:::\n" + doc for i, doc in enumerate(relevant_docs)])

    final_prompt = RAG_PROMPT_TEMPLATE.format(question=question, context=context)

    # Redact an answer
    print("=> Generating answer...")
    answer = llm(final_prompt)[0]["generated_text"]

    return answer, relevant_docs

question = "how to create a pipeline object?"

answer, relevant_docs = answer_with_rag(question, READER_LLM, KNOWLEDGE_VECTOR_DATABASE, reranker=RERANKER)

print("==================================Answer==================================")
print(f"{answer}")
print("==================================Source docs==================================")
for i, doc in enumerate(relevant_docs):
    print(f"Document {i}------------------------------------------------------------")
    print(doc)